In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from util_kelly import simulate_kelly_strategy

In [2]:
DATE_MIN = pd.Timestamp(1993, 1, 1)
DATE_MAX = pd.Timestamp(2020, 12, 31)

data = pd.read_csv('data/spx_day.csv', parse_dates=['Date'], infer_datetime_format=True, index_col=0)
data = data[data['Date'] > DATE_MIN]
data = data[data['Date'] < DATE_MAX]
data

,Date,Open,High,Low,Close,Volume
31298,1993-01-04,435.70,437.32,434.48,435.38,1.117833e+08
31299,1993-01-05,435.38,435.40,433.55,434.34,1.335278e+08
31300,1993-01-06,434.34,435.17,432.52,434.52,1.640222e+08
31301,1993-01-07,434.52,435.46,429.76,430.73,1.693611e+08
31302,1993-01-08,430.73,430.73,426.88,429.05,1.463722e+08
...,...,...,...,...,...,...
38344,2020-12-23,3693.42,3711.24,3689.28,3690.01,1.845297e+09
38345,2020-12-24,3694.03,3703.82,3689.32,3703.06,8.187849e+08
38346,2020-12-28,3723.03,3740.51,3723.03,3735.36,1.662610e+09
38347,2020-12-29,3750.01,3756.12,3723.31,3727.04,1.638287e+09


In [ ]:
k = simulate_kelly_strategy(data.copy(), rebalancing_interval=1, annual_risk_free_rate=0.02, window=252, min_kelly=0, max_kelly=100)
k.to_csv('data/spx_day_kelly.csv')

In [ ]:
fig, ax = plt.subplots(2, figsize=(12, 8), sharex=True)

ax[0].plot(np.exp(k['cum_returns']) * 100, label='Buy and Hold')
ax[0].plot(np.exp(k['strategy_cum_returns']) * 100, label='Kelly Model')
ax[0].set_ylabel('Returns (%)')
ax[0].set_title('Buy-and-hold and Long-Only Strategy with Kelly Sizing')
ax[0].legend()

ax[1].plot(k['kelly_factor'])
ax[1].set_ylabel('Leverage')
ax[1].set_xlabel('Date')
ax[1].set_title('Kelly Factor')

plt.tight_layout()
plt.show()

In [4]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [17]:
@interact(window=(0,1008,10), rebalancing_interval=(1,30,1), min_kelly=(-1,1,0.1), max_kelly=(0,100,0.1), kelly_fraction=(0,1,0.05))
def adjust_window(window=252, rebalancing_interval=1, min_kelly=0, max_kelly=3, kelly_fraction=1.0):
    k_cap = simulate_kelly_strategy(
        data.copy(),
        rebalancing_interval=rebalancing_interval,
        kelly_fraction=kelly_fraction,
        window=window,
        min_kelly=min_kelly,
        max_kelly=max_kelly
    )

    fig, ax = plt.subplots(2, figsize=(18, 8), sharex=True)

    ax[0].plot(np.exp(k_cap['cum_returns']) * 100, label='Buy and Hold')
    ax[0].plot(np.exp(k_cap['strategy_cum_returns']) * 100, label='Kelly Model')
    ax[0].set_ylabel('Returns (%)')
    ax[0].set_title('Buy-and-hold and Long-Only Strategy with Kelly Sizing')
    ax[0].legend()

    ax[1].plot(k_cap['kelly_fraction'])
    ax[1].set_ylabel('Leverage')
    ax[1].set_xlabel('Date')
    ax[1].set_title(f'Kelly Fraction (= Kelly Factor * {kelly_fraction})')

    plt.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=252, description='window', max=1008, step=10), IntSlider(value=1, descri…